In [8]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 27.1MB/s 
     |████████████████████████████████| 2.5MB 53.3MB/s 
     |████████████████████████████████| 245kB 43.1MB/s 
     |████████████████████████████████| 133kB 56.5MB/s 
     |████████████████████████████████| 3.3MB 53.3MB/s 
     |████████████████████████████████| 1.2MB 51.1MB/s 
     |████████████████████████████████| 7.8MB 21.5MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 1.8MB 51.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 901kB 47.2MB/s 
     |████████████████████████████████| 245kB 57.3MB/s 
     |████████████████████████████████| 122kB 58.8MB/s 
     |████████████████████████████████| 112kB 56.8MB/s 
     |████████████████████████████████| 4.2MB 31.2MB/s 
     |████████████████████████████████| 81kB 1.1MB/s 
     |████████████████████████████████| 174kB 59.3MB/s 
     |████████████████████████████████| 102kB 13.7MB/

In [9]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

In [10]:
train_df=pd.read_csv("/content/drive/MyDrive/KULEEAH/Semester 8/TA Farihin/Dataset/bert/train.csv")
eval_df=pd.read_csv("/content/drive/MyDrive/KULEEAH/Semester 8/TA Farihin/Dataset/bert/val.csv")

In [11]:
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
108840,1383795163412058114,Gubernur,O
108841,1383795163412058114,Khofifah,B-PER
108842,1383795163412058114,fokus,O
108843,1383795163412058114,untuk,O


In [12]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [13]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    # "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    # "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [14]:
model = NERModel('bert', 
                'bert-base-multilingual-cased',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [15]:
model.train_model(train_df,  eval_data=eval_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:739: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(722,
 {'eval_loss': [0.22709616482063963],
  'f1_score': [0.6618898343036274],
  'global_step': [722],
  'precision': [0.7140096618357488],
  'recall': [0.6168614357262103],
  'train_loss': [0.3187493085861206]})

In [11]:
result, model_outputs, wrong_preds = model.eval_model(eval_df)

Running Evaluation:   0%|          | 0/269 [00:00<?, ?it/s]

In [12]:
result

{'eval_loss': 0.22434004617778786,
 'f1_score': 0.6690761237858595,
 'precision': 0.7291974396848843,
 'recall': 0.6181135225375626}

In [6]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["Safari Ramadhan di Malang, Gubernur Khofifah ingatkan pentingnya Rembuk Nyekrup birokrat"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

NameError: ignored

In [1]:
predictions

NameError: ignored